In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv(r"C:\Users\aryan\OneDrive\Desktop\Capstone Project\Data Preprocessing New\uncleaned_gurgaon_properties-MSI.csv")

In [3]:
df.shape

(9650, 12)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [5]:
df.sample(1)

,Name,Address,Price,Area,Configuration,Floor,Facing,Overlooking,Property Age,URL,Nearby Places,details
6034,Signature Global the Millennia\n3.7,"Signature Global the Millennia\nSector 37D, Gurgaon","₹ 77 Lac+ Govt Charges & Tax\n@ 14,836 per sq.ft. (All inclusive, Negotiable) View Price Details",Super Built up area 820\n(76.18 sq.m.)\nBuilt Up area: 619 sq.ft. (57.51 sq.m.)\nCarpet area: 519 sq.ft. (48.22 sq.m.) \nsq.ft.,"2 Bedrooms , 2 Bathrooms, 3 Balconies",6th of 15 Floors,North,NaN,1 to 5 Year Old View Construction Status,https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-sale-in-signature-global-the-millennia-sector-37d-gurgaon-820-sq-ft-spid-O82523346,['The Acura BMK\nThe Esplanade Mall\nDwarka Expressway\nBrahm Dutt Blue Bells Public School\nThe Signature Advanced Super Speciality'],"Transaction Type: :, Resale | Property Ownership: :, Freehold | Flooring: :, Vitrified | Furnishing: :, Furnished | Width of facing road: :, 78.7 Feet | Gated Community: :, Yes | WheelChair Friendly: :, Yes | Pet Friendly: :, Yes | Water Source: :, Municipal corporation,24*7 Water | Power Backup: :, Partial | Property Code: :, O82523346"


In [6]:
df = df[~(df['Area'].str.contains('Plot', case= False, na= False))]

In [7]:
df.shape

(9644, 12)

**Extracting the Sector Values**

In [8]:
df['Sector'] = df['Address'].str.extract(r'Sector[-\s]+(\d+)', flags=re.IGNORECASE)

In [9]:
df['Sector'] = 'Sector ' + df['Sector']

In [10]:
# df["sector"] = df["sector"].astype("uint8")

In [11]:
df['Sector'].value_counts()

Sector
Sector 65     414
Sector 102    383
Sector 89     383
Sector 67     366
Sector 70     361
Sector 37     284
Sector 33     261
Sector 92     223
Sector 104    192
Sector 108    189
Sector 69     188
Sector 57     188
Sector 84     186
Sector 63     184
Sector 61     169
Sector 66     156
Sector 82     155
Sector 109    154
Sector 90     152
Sector 83     151
Sector 86     140
Sector 99     124
Sector 79     122
Sector 81     121
Sector 85     117
Sector 36     115
Sector 106    113
Sector 49     111
Sector 68     104
Sector 48     101
Sector 95     100
Sector 93      94
Sector 56      88
Sector 71      78
Sector 76      74
Sector 107     73
Sector 77      70
Sector 111     70
Sector 88      69
Sector 51      64
Sector 103     62
Sector 113     62
Sector 23      61
Sector 52      61
Sector 47      60
Sector 91      51
Sector 112     48
Sector 72      45
Sector 45      45
Sector 62      42
Sector 35      40
Sector 60      38
Sector 50      35
Sector 74      33
Sector 1       33
Sec

In [12]:
df['Sector'].isnull().sum()

np.int64(1874)

In [13]:
gemini_sector = pd.read_csv(r'C:\Users\aryan\OneDrive\Desktop\Capstone Project\Misc\gemini_null_sector.txt')

In [14]:
gemini_sector.drop(columns= 'Unnamed: 0', inplace= True)

In [15]:
# Make lowercase copies for matching
df['Address_lower'] = df['Address'].str.lower()
gemini_sector['Address_lower'] = gemini_sector['Address'].str.lower()

# Create a mapping from claude_sector
sector_mapping = gemini_sector.set_index('Address_lower')['Sector']

# Fill missing Sector values in df using the mapping
df['Sector'] = df.apply(
    lambda row: sector_mapping[row['Address_lower']] if pd.isna(row['Sector']) and row['Address_lower'] in sector_mapping else row['Sector'],
    axis=1
)

# Drop helper column
df.drop(columns='Address_lower', inplace=True)

In [16]:
df['Sector'].isnull().sum()

np.int64(1107)

In [17]:
gemini_sector_dict = {
  "Signature Global Park\nSohna, Gurgaon": "Sector 36",
  "Luxury Builder floor In South City 2\nSouth City 2, Gurgaon": "Sector 49",
  "Unitech Fresco\nNirvana Country, Gurgaon": "Sector 50",
  "Sushant Lok 1 Builder Floors\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "Eldeco Accolade\nSohna, Gurgaon": "Sector 2",
  "Silverglades The Melia\nSohna, Gurgaon": "Sector 35",
  "Breez Global Heights\nSohna, Gurgaon": "Sector 33",
  "GLS Arawali Homes\nSohna, Gurgaon": "Sector 4",
  "Unitech Harmony\nNirvana Country, Gurgaon": "Sector 50",
  "Ashiana Anmol\nSohna, Gurgaon": "Sector 33",
  "DLF Phase 1 Ultra Luxury Builder Floors\nDLF Phase 1, Gurgaon": "Sector 26",
  "Ashiana Mulberry\nSohna, Gurgaon": "Sector 2",
  "Luxury Builder Floor In Sushant Lok 1\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "DLF Luxury Builder Floors\nDLF Phase 2, Gurgaon": "Sector 25",
  "Suncity Vatsal Valley\nGwal Pahari, Gurgaon": "Gwal Pahari",
  "Ultra Luxury Builder Floor\nSouth City 1, Gurgaon": "Sector 41",
  "Ansals Palam Vihar\nPalam Vihar, Gurgaon": "Sector 23",
  "Signature The Serenas\nSohna, Gurgaon": "Sector 36",
  "Unitech Escape\nNirvana Country, Gurgaon": "Sector 50",
  "Uppal Southend\nUppals Southend, Gurgaon": "Sector 49",
  "The Close North\nNirvana Country, Gurgaon": "Sector 50",
  "DLF City Plots Phase 2\nDLF Phase 2, Gurgaon": "Sector 25",
  "Central Park The Orchard\nSohna, Gurgaon": "Sector 32",
  "DLF Wellington Estate\nDLF Phase 5, Gurgaon": "Sector 53",
  "HCBS Sports Ville\nSohna, Gurgaon": "Sector 35",
  "Malibu Towne\nMalibu Town, Gurgaon": "Sector 47",
  "Eldeco Acclaim\nSohna, Gurgaon": "Sector 2",
  "DLF Builder Floors\nDLF Phase 2, Gurgaon": "Sector 25",
  "The Close South\nNirvana Country, Gurgaon": "Sector 50",
  "Bestech Park View Residency\nPalam Vihar, Gurgaon": "Sector 3",
  "Trisara Our Homes 3\nSohna, Gurgaon": "Sector 6",
  "DLF Regency Park 2\nDLF Phase 4, Gurgaon": "Sector 27",
  "MVN Athens\nSohna, Gurgaon": "Sector 5",
  "DLF Oakwood Estate\nDLF Phase 2, Gurgaon": "Sector 25",
  "DLF Carlton Estate\nDLF Phase 5, Gurgaon": "Sector 53",
  "Unitech Singleton Floors\nSouth City 2, Gurgaon": "Sector 49",
  "DLF Belvedere Towers\nDLF Phase 2, Gurgaon": "Sector 24",
  "Ardee Palm Grove Heights\nArdee City, Gurgaon": "Sector 52",
  "Central Park Flower Valley The Room\nSohna, Gurgaon": "Sector 32",
  "DLF Ridgewood Estate\nDLF Phase 4, Gurgaon": "Sector 27",
  "Bedarwal Consortium Sixty Three Golf Drive\nKadarpur, Gurgaon": "Sector 63",
  "DLF Phase 4 Luxury Builder Floors\nDLF Phase 4, Gurgaon": "Sector 27",
  "Ultra Luxurious Builder Floor\nDLF Phase 1, Gurgaon": "Sector 26",
  "DLF Regency Park 1\nDLF Phase 4, Gurgaon": "Sector 27",
  "Brand new ultra luxury builder floor\nDLF Phase 1, Gurgaon": "Sector 26",
  "DLF Belvedere Park\nDLF Phase 2, Gurgaon": "Sector 24",
  "Unitech Heritage City\nHeritage City, Gurgaon": "Sector 25",
  "Ansal Sushant Residency\nSushant Lok Phase 3, Gurgaon": "Sector 57",
  "Private Builder Floor\nUppals Southend, Gurgaon": "Sector 49",
  "Woodstock Floors\nNirvana Country, Gurgaon": "Sector 50",
  "Ultra Luxury Builder floor In Nirvana Country\nNirvana Country, Gurgaon": "Sector 50",
  "DLF Exclusive Floors\nDLF Phase 5, Gurgaon": "Sector 53",
  "Signature Global Park 4\nSohna, Gurgaon": "Sector 36",
  "sushant lok- 3\nSushant Lok 3 Extension, Gurgaon": "Sector 57",
  "Ardee The Residency\nArdee City, Gurgaon": "Sector 52",
  "Valley View Estate\nGwal Pahari, Gurgaon": "Gwal Pahari",
  "Nirvana Cedar Crest\nNirvana Country, Gurgaon": "Sector 50",
  "Ansal\nBlock C, Sushant Lok Phase 1, Gurgaon": "Sector 43",
  "Sushant Lok 1 Builder Floors\nBlock C, Sushant Lok Phase 1, Gurgaon": "Sector 43",
  "Emaar MGF The Enclave\nBadshahpur, Gurgaon": "Sector 66",
  "Emerald Floors Premier Phase 3\nGolf Course Road, Gurgaon": "Sector 65",
  "Garden Villas\nDLF Phase 4, Gurgaon": "Sector 27",
  "Breez Global Hill View\nSohna, Gurgaon": "Sector 33",
  "Sushant lok 3\nSushant Lok 3 Extension, Gurgaon": "Sector 57",
  "Independent Floor at DLF City\nDLF Phase 3, Gurgaon": "Sector 24",
  "Ardee City\nArdee City, Gurgaon": "Sector 52",
  "Ansal\nSushant Lok Phase 2, Gurgaon": "Sector 56",
  "Malibu Towers\nMalibu Town, Gurgaon": "Sector 47",
  "Independent Floors at DLF City Phase 4\nDLF Phase 4, Gurgaon": "Sector 27",
  "Smart World Orchard\nUllahawas, Gurgaon": "Sector 61",
  "Sushant Lok 1 Builder Floors\nB Block Sushant Lok Phase 1, Gurgaon": "Sector 43",
  "TOP FLOOR TERRACE GARDEN BAR AND KITCHENS PANTRY\nDLF Phase 2, Gurgaon": "Sector 25",
  "Private Builder Floor\nMayfield Garden, Gurgaon": "Sector 51",
  "Global Foyer\nPalam Vihar, Gurgaon": "Sector 23",
  "ASF Isle De Royale\nGwal Pahari, Gurgaon": "Gwal Pahari",
  "Landmark Avenue\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "Godrej Aria\nNaurangpur, Gurgaon": "Sector 79",
  "DLF Princeton Estate\nDLF Phase 5, Gurgaon": "Sector 53",
  "DLF Floors Phase 3\nDLF Phase 3, Gurgaon": "Sector 24",
  "Ambience Lagoon\nDLF Phase 3, Gurgaon": "Sector 24",
  "DLF Pink Town House\nDLF Phase 3, Gurgaon": "Sector 24",
  "Super Luxury Mindblowing\nDLF Phase 2, Gurgaon": "Sector 25",
  "Super Luxury Mind Blowing\nDLF Phase 2, Gurgaon": "Sector 25",
  "PARK FACING LUXURY BUILDER FLOOR\nDLF Phase 1, Gurgaon": "Sector 26",
  "Paramount Luxury Floors 2\nMalibu Town, Gurgaon": "Sector 47",
  "Apexhomz\nPalam Vihar, Gurgaon": "Sector 23",
  "BUILDER FLOOR\nDLF Phase 2, Gurgaon": "Sector 25",
  "brand new upper ground floor\nDLF Phase 2, Gurgaon": "Sector 25",
  "Top terrace old floor\nDLF Phase 1, Gurgaon": "Sector 26",
  "middle floor\nDLF Phase 2, Gurgaon": "Sector 25",
  "Upper ground Floor\nBlock C, Sushant Lok Phase 1, Gurgaon": "Sector 43",
  "K BLOCK\nDLF Phase 2, Gurgaon": "Sector 25",
  "Builder floor\nDLF Cyber City, Gurgaon": "Sector 24",
  "Bestech Park View City\nPark View City 1, Gurgaon": "Sector 48",
  "The Close North\nPocket H Nirvana Country, Gurgaon": "Sector 50",
  "DLF Regent House\nDLF Phase 4, Gurgaon": "Sector 27",
  "Ansals Palam Vihar\nBlock C 1 Palam Vihar, Gurgaon": "Sector 23",
  "Emaar MGF The Palm Drive\nBadshahpur, Gurgaon": "Sector 66",
  "builder fl\nDLF Cyber City, Gurgaon": "Sector 24",
  "DLF Luxury Builder Floors\nBlock M DLF phase 2, Gurgaon": "Sector 25",
  "Ultra Luxury Builder Floor Mayfield Garden Gurgaon\nB Block, Mayfield Garden, Gurgaon": "Sector 51",
  "Luxury Builder Floor\nB Block, Mayfield Garden, Gurgaon": "Sector 51",
  "Ultra Luxury Floor\nDLF Phase 2, Gurgaon": "Sector 25",
  "BRAND NEW CONSTRUCTION\nDLF Phase 2, Gurgaon": "Sector 25",
  "mayfield garden\nBlock K, Mayfield Garden, Gurgaon": "Sector 51",
  "Designer Luxurious Floor\nBlock C, Sushant Lok Phase 1, Gurgaon": "Sector 43",
  "Shiv Shankar Society\nSamaspur, Gurgaon": "Sector 51",
  "E-16 DLF\nBlock E DLF City Phase 1, Gurgaon": "Sector 26",
  "ultra luxury builder floor\nDLF Phase 2, Gurgaon": "Sector 25",
  "DLF, Block-J\nDLF Phase 2, Gurgaon": "Sector 25",
  "Builder Floor Brand New Construction\nDLF Phase 2, Gurgaon": "Sector 25",
  "Luxury Builder Floor\nBlock C South city 1, Gurgaon": "Sector 41",
  "Luxury Builder Floor\nSouth City 1, Gurgaon": "Sector 41",
  "Luxury DLF City Floors\nDLF Phase 1, Gurgaon": "Sector 26",
  "Ultra Luxury Builder Floor In Mayfield Garden\nMayfield Garden, Gurgaon": "Sector 51",
  "Greenwood City Luxurious Builder Floor\nGreenwood City, Gurgaon": "Sector 45",
  "Unitech Deerwood Chase\nNirvana Country, Gurgaon": "Sector 50",
  "Unitech Espace\nNirvana Country, Gurgaon": "Sector 50",
  "Belvedere Towers\nDLF Cyber City, Gurgaon": "Sector 24",
  "ansal\nPalam Vihar, Gurgaon": "Sector 23",
  "sushant lok- 3\nSushant Lok 3 Extension, Gurgaon": "Sector 57",
  "mayfield Garden\nC Block Mayfield Garden, Gurgaon": "Sector 51",
  "DLF The Grove\nDLF Phase 5, Gurgaon": "Sector 53",
  "Ansal Sushant Lok Plots\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "BRAND NEW SUPER LUXURY BUILDER FLOOR\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "Shibha Realty\nDLF Phase 2, Gurgaon": "Sector 25",
  "Sushant lok-1 Block A\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "Unitech Fresco\nPocket H Nirvana Country, Gurgaon": "Sector 50",
  "18MTR ROAD LUXURY BUILDER FLOOR\nDLF Phase 1, Gurgaon": "Sector 26",
  "AVENUE-49\nSouth City 2, Gurgaon": "Sector 49",
  "Namo Homes ultra luxury independent builder floor\nDLF Phase 1, Gurgaon": "Sector 26",
  "DLF The Skycourt\nNawada Fatehpur, Gurgaon": "Sector 86",
  "M block\nDLF Cyber City, Gurgaon": "Sector 24",
  "Ansal floor Sushant Lok 3\nSushant Lok Phase 3, Gurgaon": "Sector 57",
  "Builder floor Apartment\nSushant Lok 3 Extension, Gurgaon": "Sector 57",
  "Super Luxury Mind Blowing\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "Builder Floor\nBlock C Ardee City, Gurgaon": "Sector 52",
  "Super Luxury Mind Blowing\nDLF Phase 4, Gurgaon": "Sector 27",
  "215SQYD BEST AND CHEAPEST PRICE OPTION\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "DLF-1 ARJUN MARG SUPER LUXURY FLOOR 300SQYD\nDLF Phase 1, Gurgaon": "Sector 26",
  "Luxury Builder floor In South City 2\nDLF Phase 3, Gurgaon": "Sector 24",
  "South City2\nSouth City 2, Gurgaon": "Sector 49",
  "Conscient Heritage One\nHeritage City, Gurgaon": "Sector 25",
  "HCS The Leaf\nGwal Pahari, Gurgaon": "Gwal Pahari",
  "Ansal Harmony Homes\nSushant Lok Phase 3, Gurgaon": "Sector 57",
  "Suncity Luxury Builder Floors\nSuncity, Gurgaon": "Sector 54",
  "k block\nDLF Phase 2, Gurgaon": "Sector 25",
  "Ansal Palam Vihar\nPalam Vihar, Gurgaon": "Sector 23",
  "super luxury floor\nDLF Phase 4, Gurgaon": "Sector 27",
  "Namo homes Ultra luxury builder floor\nDLF Phase 4, Gurgaon": "Sector 27",
  "M Block Top Terrace\nDLF Phase 2, Gurgaon": "Sector 25",
  "C block\nBlock C, Sushant Lok Phase 1, Gurgaon": "Sector 43",
  "The Retreat\nSouth City 1, Gurgaon": "Sector 41",
  "C-1 Palam vihar\nBlock C 1 Palam Vihar, Gurgaon": "Sector 23",
  "Apartment.\nWazirpur, Gurgaon": "Sector 99",
  "Brand New Builder Floor\nBlock C, Sushant Lok Phase 1, Gurgaon": "Sector 43",
  "Top Floor Terrace\nDLF Phase 1, Gurgaon": "Sector 26",
  "Ansal Oriental Villas\nSushant Lok Phase 3, Gurgaon": "Sector 57",
  "middle floor luxury floor\nDLF Phase 3, Gurgaon": "Sector 24",
  "Middle Floor Luxury\nDLF Phase 2, Gurgaon": "Sector 25",
  "Super Luxury Corner Middle floor\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "CORNER GREEN BELT FLOOR\nDLF Phase 1, Gurgaon": "Sector 26",
  "Builder Floor Dlf 4\nDLF Phase 4, Gurgaon": "Sector 27",
  "Corner ultra luxury builder floor\nDLF Phase 1, Gurgaon": "Sector 26",
  "TWO SIDE OPEN FLOOR\nDLF Phase 2, Gurgaon": "Sector 25",
  "TOP FLOOR TERRACE GARDEN BAR AND KITCHENS\nDLF Phase 2, Gurgaon": "Sector 25",
  "LUXURIOUS BUILDER FLOOR\nDLF Phase 2, Gurgaon": "Sector 25",
  "BEST PRICE DEAL LUXURY BUILDER FLOOR\nDLF Phase 2, Gurgaon": "Sector 25",
  "Jai maa vaishno homes\nNew Colony, Gurgaon": "Sector 7",
  "Swastik apartment\nDharam Colony, Gurgaon": "Sector 109",
  "Vipul Orchid Gardens\nSuncity, Gurgaon": "Sector 54",
  "Himalayan Society\nManesar, Gurgaon": "Manesar",
  "The Crew Bos Chs Manesar\nManesar, Gurgaon": "Manesar",
  "Valley View estate\nGwal Pahari, Gurgaon": "Gwal Pahari",
  "Mayfield garden\nC Block Mayfield Garden, Gurgaon": "Sector 51",
  "Sushant lok\nG Block, Sushant Lok 3, Gurgaon": "Sector 57",
  "on request\nPalam Vihar, Gurgaon": "Sector 23",
  "DLF Phase 1Brand New Ultra Luxury Builder Floors\nBlock H, DLF City Phase 1, Gurgaon": "Sector 26",
  "Super Luxury North Facing\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "215SQYD PARK FACING SUPER LUXURY\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "The Good Luck CGHS\nWazirabad, Gurgaon": "Sector 56",
  "CGHS Aravali Homes\nSuncity, Gurgaon": "Sector 54",
  "Bestech Center Point\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "Signature Global Park 5\nSohna, Gurgaon": "Sector 36",
  "serenas from signature global\nSohna Road, Gurgaon": "Sector 36",
  "Ansal Royale Casa\nSushant Lok Phase 2, Gurgaon": "Sector 56",
  "Radhika Apartments\nNew Palam Vihar Phase 3, Gurgaon": "Sector 110",
  "The New Greenwood CGHS\nWazirabad, Gurgaon": "Sector 56",
  "Mittals Royale Orchards\nBinola, Gurgaon": "Binola",
  "Signature Global Orchard Avenue 2\nHayatpur, Gurgaon": "Sector 93",
  "Divya Apartments\nDayanand Colony, Gurgaon": "Sector 6",
  "Hariom Apartment\nRam Vihar, Gurgaon": "Sector 104",
  "Ompee Green\nNew Palam Vihar, Gurgaon": "Sector 110",
  "Vastu Apartment\nSushant Lok Phase 2, Gurgaon": "Sector 56",
  "exclusive floors owners society\nGolf Course Road, Gurgaon": "Sector 54",
  "Silver Oaks Estates\nDLF Phase 1, Gurgaon": "Sector 26",
  "Shri Sai Apartments\nSai Kunj Block C, Gurgaon": "Sector 17",
  "Green Court\nNew Gurgaon": "Sector 90",
  "Ansals API Esencia\nGolf Course Ext Road, Gurgaon": "Sector 67",
  "Signature Global\nWazirpur, Gurgaon": "Sector 99",
  "Tulip Garden, Gurgaon\nSushant Lok Phase 2, Gurgaon": "Sector 56",
  "Ultra Luxury Builder Floor\nBlock H, DLF City Phase 1, Gurgaon": "Sector 26",
  "aarambh buildtech\nDharam Colony, Gurgaon": "Sector 109",
  "Ansal Sushant Apartments\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "Oriental Homes\nSushant Lok Phase 3, Gurgaon": "Sector 57",
  "Ansal Homes\nSushant Lok Phase 3, Gurgaon": "Sector 57",
  "Anubhav Apartment\nAshok Vihar, Gurgaon": "Sector 3",
  "AEZ Aloha\nSushant Lok 3 Extension, Gurgaon": "Sector 57",
  "builder floor\nB Block Sushant Lok Phase 1, Gurgaon": "Sector 43",
  "Sushant lok-3\nSushant Lok Phase 3, Gurgaon": "Sector 57",
  "avenue 49\nSouth City 2, Gurgaon": "Sector 49",
  "Brand New Builder Floor\nDLF Phase 1, Gurgaon": "Sector 26",
  "Tani Homes\nRajendra Park, Gurgaon": "Sector 105",
  "hno- A 34\nBlock A South City 2, Gurgaon": "Sector 49",
  "Lakshman Vihar Phase 2 gali na 10\nLaxman Vihar Phase 2, Gurgaon": "Sector 3",
  "Genesis Sky heights\nBhiwadi, Gurgaon": "Bhiwadi",
  "Suncity Township\nGolf Course Road, Gurgaon": "Sector 54",
  "Nirmal Towers N-406\nBlock N, New Palam Vihar Phase 1, Gurgaon": "Sector 110",
  "Luxury Builder floor In South City 2\nBlock H South city 2, Gurgaon": "Sector 49",
  "Ansal Florence Super\nSushant Lok 3 Extension, Gurgaon": "Sector 57",
  "LUXURY MIDDLE FLOOR\nDLF Phase 2, Gurgaon": "Sector 25",
  "BEST PRICE DEAL LUXURY BUILDER FLOOR\nDLF Phase 1, Gurgaon": "Sector 26",
  "Spaze Privy AT4\nSihi Village, Gurgaon": "Sector 84",
  "Sushant lok\nSushant Lok Phase 2, Gurgaon": "Sector 56",
  "DLF-2 3BHK BEST AND CHEAPEST OPTION\nDLF Phase 2, Gurgaon": "Sector 25",
  "Today Luxotica\nMayfield Garden, Gurgaon": "Sector 51",
  "Sushant lok\nSushant Lok Phase 3, Gurgaon": "Sector 57",
  "Sushant lok 3\nBlock B Sushant Lok 3, Gurgaon": "Sector 57",
  "DLF-2 ULTRA LUXURIOUS FLOOR CHEAPEST PRICE\nDLF Phase 2, Gurgaon": "Sector 25",
  "sushant lok 3\nSushant Lok 3 Extension, Gurgaon": "Sector 57",
  "F block Dlf phase 1\nGolf Course Road, Gurgaon": "Sector 28",
  "rwa j block palam vihar Gurgaon\nPocket J Palam Vihar, Gurgaon": "Sector 2",
  "Riddhi Siddhi floor\nMayfield Garden, Gurgaon": "Sector 51",
  "Upper Ground Floor\nBlock H, DLF City Phase 1, Gurgaon": "Sector 26",
  "gdge\nNew Colony, Gurgaon": "Sector 7",
  "M3M Cosmopolitan\nGolf Course Ext Road, Gurgaon": "Sector 66",
  "Golden Floors\nMayfield Garden, Gurgaon": "Sector 51",
  "Paramount Super Luxury Floors 2\nNirvana Country, Gurgaon": "Sector 50",
  "Private Builder Floor\nRosewood City, Gurgaon": "Sector 49",
  "PRIVATE BUILDER FLOOR\nRosewood City, Gurgaon": "Sector 49",
  "Central Park Flower Valley Clover Floors\nSohna, Gurgaon": "Sector 32",
  "300SQYD BEST AND CHEAPEST PRICE DEAL\nDLF Phase 2, Gurgaon": "Sector 25",
  "DLF-2 215SQYD FLOOR CHEAPEST OPTION\nDLF Phase 2, Gurgaon": "Sector 25",
  "DLF-2 LUXURIOUS FLOOR CHEAPEST DEAL\nDLF Phase 2, Gurgaon": "Sector 25",
  "Top floor Terrace\nDLF Phase 1, Gurgaon": "Sector 26",
  "DLF South City 1\nSouth City 1, Gurgaon": "Sector 41",
  "3 side open\nDLF Phase 4, Gurgaon": "Sector 27",
  "PARK FACING LUXURY BUILDER FLOOR\nDLF Phase 2, Gurgaon": "Sector 25",
  "BEST PRICE DEAL\nDLF Phase 2, Gurgaon": "Sector 25",
  "PARK FACING EAST FACING\nDLF Phase 2, Gurgaon": "Sector 25",
  "AKASHNEEM MARG\nDLF Phase 2, Gurgaon": "Sector 25",
  "w block uppal southend\nUppals Southend, Gurgaon": "Sector 49",
  "ARJUN MARG\nDLF Phase 1, Gurgaon": "Sector 26",
  "M3M Antalya Hills\nNaurangpur, Gurgaon": "Sector 79",
  "M Block Dlf City-2\nDLF Phase 2, Gurgaon": "Sector 25",
  "U Block White Town House\nDLF Phase 3, Gurgaon": "Sector 24",
  "Brand New Builder Floor\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "Best price uxury floor\nDLF Phase 2, Gurgaon": "Sector 25",
  "DLF City Plots Phase 2\nBlock M DLF phase 2, Gurgaon": "Sector 25",
  "BEST LAYOUT LUXURY FLOOR\nDLF Phase 2, Gurgaon": "Sector 25",
  "TOP FLOOR HAVING TERRACE\nDLF Phase 2, Gurgaon": "Sector 25",
  "Flora Avenue 33\nSohna, Gurgaon": "Sector 33",
  "G block south city\nSouth City 2, Gurgaon": "Sector 49",
  "Best price luxury floor\nDLF Phase 4, Gurgaon": "Sector 27",
  "Builder Floor\nDLF Phase 1, Gurgaon": "Sector 26",
  "Brand New Ultra Luxury Builder Floor\nDLF Phase 1, Gurgaon": "Sector 26",
  "Builder Floor Top\nDLF Phase 2, Gurgaon": "Sector 25",
  "Brand new ultra luxury builder floor\nDLF Phase 4, Gurgaon": "Sector 27",
  "huda builder floor\nUrban Estate, Gurgaon": "Sector 4",
  "418SQYD LOW BUDGET FLOOR\nSushant Lok Phase 1, Gurgaon": "Sector 43",
  "builder floor\nBlock C Palam Vihar, Gurgaon": "Sector 23",
  "Best price luxury floor\nDLF Phase 2, Gurgaon": "Sector 25",
  "Park Facing luxury floor\nDLF Phase 2, Gurgaon": "Sector 25",
  "VALUE FOR MONEY DEAL 300SQYD\nDLF Phase 2, Gurgaon": "Sector 25",
  "SS Aaron Ville\nUppals Southend, Gurgaon": "Sector 49",
  "CGHS Satguru Apartments\nWazirabad, Gurgaon": "Sector 56"
}

In [18]:
gemini_sector_2 = pd.DataFrame(gemini_sector_dict.items(), columns=['Address', 'Sector'])

In [ ]:
# Make lowercase copies for matching
df['Address_lower'] = df['Address'].str.lower()
gemini_sector_2['Address_lower'] = gemini_sector_2['Address'].str.lower()

# Create a mapping from claude_sector
sector_mapping = gemini_sector_2.set_index('Address_lower')['Sector']

# Fill missing Sector values in df using the mapping
df['Sector'] = df.apply(
    lambda row: sector_mapping[row['Address_lower']] if pd.isna(row['Sector']) and row['Address_lower'] in sector_mapping else row['Sector'],
    axis=1
)

# Drop helper column
df.drop(columns='Address_lower', inplace=True)


In [20]:
df['Sector'].isnull().sum()

np.int64(0)

In [21]:
df['Sector'] = df['Sector'].astype(str)

In [22]:
df['Sector'] = df['Sector'].str.replace('\n', ' ', regex=False).str.strip()

In [23]:
df['Sector'].dtype

dtype('O')

In [ ]:
def clean_sector(val):
    # Remove anything starting from 'Address_lower' and remove trailing 'Name: Sector, dtype: object'
    if 'Name: Sector' in val:
        return re.sub(r'Address_lower.*?(Sector \d+|[A-Za-z ]+?) .*?Name: Sector, dtype: object', r'\1', val)
    return val

df['Sector'] = df['Sector'].apply(clean_sector)

In [32]:
df['Sector'].value_counts().reset_index().sort_values(by= 'Sector', ascending= False)

,Sector,count
105,Wazirabad,1
70,Unknown,24
104,Sohna Road,1
26,Sector 99,126
37,Sector 95,100
40,Sector 93,95
10,Sector 92,223
55,Sector 91,51
23,Sector 90,153
75,Sector 9,14


In [41]:
temp_df = df[~df['Sector'].astype(str).str.contains(r'\bSector\b', case=False, na=False)][['Address', 'Sector']]

In [42]:
temp_df.to_csv(r'C:\Users\aryan\OneDrive\Desktop\Capstone Project\Misc\Leftover 90 sectors.csv')

In [44]:
new_dict = {
    "Huda Sector, Gurgaon": "Sector 54",
    "Dharam Colony, Gurgaon": "Sector 23",
    "Suncity Vatsal Valley\nGwal Pahari, Gurgaon": "Sector 55",
    "Mianwali Colony, Gurgaon": "Sector not found",
    "Sohna, Gurgaon": "Sector not found",
    "K BLOCK\nDLF Phase 2, Gurgaon": "Sector not found",
    "Sushant lok 3\nSushant Lok 3 Extension, Gurgaon": "Sector not found",
    "DLF Cyber City, Gurgaon": "Sector not found",
    "Pragati Hills Society, Gurgaon": "Sector not found",
    "Ramgarh, Gurgaon": "Sector not found",
    "Urban Estate, Gurgaon": "Sector not found",
    "mayfield Garden\nC Block Mayfield Garden, Gurgaon": "Sector not found",
    "Huda, Gurgaon": "Sector 23",
    "Valley View Estate\nGwal Pahari, Gurgaon": "Sector not found",
    "Brand new ultra luxury builder floor\nDLF Phase 1, Gurgaon": "Sector not found",
    "Top Floor Terrace\nDLF Phase 1, Gurgaon": "Sector 42",
    "DLF CITY, Gurgaon": "Sector not found",
    "Mayfield garden\nC Block Mayfield Garden, Gurgaon": "Sector not found",
    "Koyal Vihar, Gurgaon": "Sector not found",
    "Himalayan Society\nManesar, Gurgaon": "Sector not found",
    "The Crew Bos Chs Manesar\nManesar, Gurgaon": "Sector not found",
    "Valley View estate\nGwal Pahari, Gurgaon": "Sector not found",
    "ASF Isle De Royale\nGwal Pahari, Gurgaon": "Sector 55",
    "Mittals Royale Orchards\nBinola, Gurgaon": "Sector not found",
    "Block K1 New Palam Vihar Phase 1, Gurgaon": "Sector 23",
    "Wazirabad, Gurgaon": "Sector not found",
    "Genesis Sky heights\nBhiwadi, Gurgaon": "Sector not found",
    "Laxman Vihar, Gurgaon": "Sector not found",
    "Vishnu Garden, Gurgaon": "Sector not found",
    "Mohyal Colony, Gurgaon": "Sector not found",
    "Dwarka Expressway, Gurgaon": "Sector not found",
    "sushant lok 3\nSushant Lok 3 Extension, Gurgaon": "Sector not found",
    "Gwal Pahari, Gurgaon": "Sector not found",
    "Private Builder Floor\nRosewood City, Gurgaon": "Sector not found",
    "PRIVATE BUILDER FLOOR\nRosewood City, Gurgaon": "Sector not found",
    "Top floor Terrace\nDLF Phase 1, Gurgaon": "Sector not found",
    "Block A New Palam Vihar Phase 2, Gurgaon": "Sector 23",
    "Brand New Ultra Luxury Builder Floor\nDLF Phase 1, Gurgaon": "Sector not found",
    "HCS The Leaf\nGwal Pahari, Gurgaon": "Sector not found",
    "Patel Nagar, Gurgaon": "Sector not found",
    "k block\nDLF Phase 2, Gurgaon": "Sector not found",
    "Jyoti Park, Gurgaon": "Sector not found"
}

In [45]:
gemini_sector_3 = pd.DataFrame(new_dict.items(), columns=['Address', 'Sector'])

# Make lowercase copies for matching
df['Address_lower'] = df['Address'].str.lower()
gemini_sector_3['Address_lower'] = gemini_sector_3['Address'].str.lower()

# Create a mapping from claude_sector
sector_mapping_3 = gemini_sector_3.set_index('Address_lower')['Sector']

# Fill missing Sector values in df using the mapping
df['Sector'] = df.apply(
    lambda row: sector_mapping_3[row['Address_lower']] if pd.isna(row['Sector']) and row['Address_lower'] in sector_mapping else row['Sector'],
    axis=1
)

# Drop helper column
df.drop(columns='Address_lower', inplace=True)

In [56]:
df['Sector'] = df['Sector'].str.strip()

In [227]:
df['Sector'].value_counts()

Sector
Sector 65      417
Sector 89      383
Sector 102     383
Sector 67      368
Sector 70      361
Sector 33      318
Sector 57      308
Sector 49      289
Sector 37      284
Sector 43      253
Sector 92      223
Sector 104     193
Sector 63      191
Sector 36      189
Sector 108     189
Sector 69      188
Sector 84      187
Sector 50      178
Sector 61      171
Sector 66      161
Sector 2       157
Sector 109     156
Sector 82      155
Sector 90      153
Sector 83      151
Sector 86      141
Sector 99      126
Sector 79      125
Sector 51      123
Sector 26      122
Sector 81      121
Sector 85      117
Sector 106     113
Sector 47      106
Sector 25      105
Sector 68      104
Sector 48      103
Sector 52      103
Sector 95      100
Sector 56       97
Sector 93       95
Sector 23       95
Sector 35       84
Sector 28       83
Sector 71       78
Sector 76       74
Sector 107      73
Sector 77       70
Sector 111      70
Sector 88       69
Sector 27       66
Sector 103      62
Secto

In [87]:
df.loc[df['Sector'] == 'mayfield', 'Sector'] = 'Sector 51'

In [225]:
df[df['Sector'] == 'Unknown']

,Name,Address,Price,Area,Configuration,Floor,Facing,Overlooking,Property Age,URL,Nearby Places,details,Sector
9292,"Patel Nagar, Gurgaon","Patel Nagar, Gurgaon","₹ 85.36 Lac+ Govt Charges & Tax\n@ 6,566 per sq.ft. (All inclusive, Negotiable)",Super Built up area 1300\n(120.77 sq.m.)\nBuilt Up area: 1250 sq.ft. (116.13 sq.m.)\nCarpet area: 1200 sq.ft. (111.48 sq.m.) \nsq.ft.,"3 Bedrooms , 2 Bathrooms, 1 Balcony",2nd of 4 Floors,East,"Park/Garden,Others,Main Road",0 to 1 Year Old,https://www.99acres.com/3-bhk-bedroom-independent-builder-floor-for-sale-in-patel-nagar-gurgaon-1300-sq-ft-spid-J82580132,['Hanuman Mandir\nState bank of india ATM\nHdfc bank ATM\nIcici bank ATM\nAhmed Hospital Multi Speciality\nDayal Eye & Maternity Centre'],"Property Ownership: :, Freehold | Flooring: :, Vitrified | Furnishing: :, Semifurnished | Width of facing road: :, 20.0 Feet | Gated Community: :, Yes | Corner Property: :, Yes | Parking: :, 2 Covered, 1 Open | WheelChair Friendly: :, Yes | Pet Friendly: :, Yes | Water Source: :, Borewell/Tank,24*7 Water,Municipal corporation | Power Backup: :, Partial | Property Code: :, J82580132",Unknown


In [222]:
# df.drop(index= 6347, inplace= True)
df.loc[df['URL'] == 'https://www.99acres.com/2-bhk-bedroom-independent-builder-floor-for-sale-in-jyoti-park-gurgaon-1200-sq-ft-spid-T83051630', 'Sector'] = 'Sector 7'

In [226]:
df.drop(index= 9292, inplace= True)

In [228]:
df.to_csv(r'C:\Users\aryan\OneDrive\Desktop\Capstone Project\Data Preprocessing New\uncleaned_gurgaon_properties_v1.csv')

In [11]:
df = df[['Name', 'Address', 'Sector', 'Price', 'Area', 'Configuration', 'Floor', 'Facing', 'Overlooking', 'Property Age', 'URL', 'Nearby Places', 'details']]

In [12]:
df.columns

Index(['Name', 'Address', 'Sector', 'Price', 'Area', 'Configuration', 'Floor',
       'Facing', 'Overlooking', 'Property Age', 'URL', 'Nearby Places',
       'details'],
      dtype='object')

In [13]:
df.sample(1)

,Name,Address,Sector,Price,Area,Configuration,Floor,Facing,Overlooking,Property Age,URL,Nearby Places,details
6659,Shree Vardhman Mantra\n3.8,"Shree Vardhman Mantra\nSector 67, Gurgaon",67,"₹ 78 Lac+ Govt Charges & Tax\n@ 9,750 per sq.ft. (All inclusive, Negotiable) View Price Details",Super Built up area 800\n(74.32 sq.m.)\nCarpet area: 520 sq.ft. (48.31 sq.m.) \nsq.ft.,"2 Bedrooms , 2 Bathrooms, 1 Balcony with Others",11st of 13 Floors,North,"Pool,Main Road,Sea facing,Park/Garden,Others,Club",1 to 5 Year Old,https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-sale-in-shree-vardhman-mantra-sector-67-gurgaon-800-sq-ft-r2-spid-V77528769,['Sector 55-56 Metro Station\nAiria Mall\nEuro International School\nGurugram University\nPulse Multispecility Hospital'],"Transaction Type: :, Resale | Property Ownership: :, Freehold | Flooring: :, Vitrified | Furnishing: :, Semifurnished | Width of facing road: :, 10.0 Feet | Gated Community: :, Yes | Corner Property: :, Yes | Parking: :, 1 Covered, 1 Open | WheelChair Friendly: :, Yes | Pet Friendly: :, Yes | Water Source: :, Municipal corporation,24*7 Water,Borewell/Tank | Power Backup: :, Full | Property Code: :, V77528769"


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9644 entries, 0 to 9649
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           9639 non-null   object
 1   Address        9644 non-null   object
 2   Sector         7770 non-null   object
 3   Price          9644 non-null   object
 4   Area           9644 non-null   object
 5   Configuration  9644 non-null   object
 6   Floor          9644 non-null   object
 7   Facing         8692 non-null   object
 8   Overlooking    8751 non-null   object
 9   Property Age   9642 non-null   object
 10  URL            9644 non-null   object
 11  Nearby Places  9488 non-null   object
 12  details        9644 non-null   object
dtypes: object(13)
memory usage: 1.0+ MB


In [21]:
# df[df['Sector'].isnull()][['Name', 'Address', 'Sector']]
df[df['Sector'].isnull()]['Address'].value_counts().reset_index().to_csv(r'C:\Users\aryan\OneDrive\Desktop\Capstone Project\Misc\Null Sector Address.csv')

**Name**

In [135]:
df['Name'].value_counts()

Name
Sobha City\n3.7                                      223
Shapoorji Pallonji Joyville Gurugram\n3.9            189
M3M Heights\n3.8                                     144
Smart World Gems                                     128
M3M Soulitude                                         98
DLF Phase 2, Gurgaon                                  97
Hero Homes                                            95
Smart World Orchard\n3.6                              94
Suncity Avenue 76\n3.8                                86
Sushant Lok Phase 1, Gurgaon                          85
DLF Phase 1, Gurgaon                                  80
Emaar MGF The Palm Drive\n4.0                         80
Ireo The Corridors\n3.9                               77
Palam Vihar, Gurgaon                                  77
DLF The Ultima\n4.2                                   74
Sare Homes                                            73
DLF New Town Heights 1\n4.1                           68
Pyramid Urban\n3.9        

In [136]:
df[['Property Name', 'Rating']] = df['Name'].str.split('\n', expand= True)
df['Rating'] = df['Rating'].replace('', np.nan)
df['Rating'] = df['Rating'].astype(float)

In [137]:
df.sample(1)

,Name,Address,sector,Price,Area,Configuration,Floor,Facing,Overlooking,Property Age,URL,Nearby Places,details,Property Name,Rating
634,Ramprastha The Edge Towers\n3.5,"Ramprastha The Edge Towers\nSector 37D, Gurgaon",37,"₹ 1.52 Crore+ Govt Charges & Tax\n@ 12,063 per sq.ft. (All inclusive) View Price Details",Super Built up area 1770\n(164.44 sq.m.)\nCarpet area: 1260 sq.ft. (117.06 sq.m.) \nsq.ft.,"3 Bedrooms , 3 Bathrooms, 3+ Balconies",12nd of 19 Floors,North,Park/Garden,1 to 5 Year Old,https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-sale-in-ramprastha-the-edge-towers-sector-37d-gurgaon-1770-sq-ft-r5-spid-Y72300566,['The Esplanade Mall\nDwarka Express\nEuro International School\nNarayana Junior College\nYashroop Hospital \nIndira Gandhi International Airport'],"Transaction Type: :, Resale | Property Ownership: :, Freehold | Flooring: :, Vitrified | Furnishing: :, Semifurnished | Gated Community: :, Yes | Parking: :, 1 Covered | WheelChair Friendly: :, Yes | Pet Friendly: :, Yes | Water Source: :, Municipal corporation,Borewell/Tank,24*7 Water | Power Backup: :, Full | Property Code: :, Y72300566",Ramprastha The Edge Towers,3.5


In [138]:
df['Name'].isnull().sum()

np.int64(300)

In [139]:
df['Property Name'].isnull().sum()

np.int64(300)

In [140]:
df['Rating'].isnull().sum()

np.int64(3673)

In [141]:
df = df[['Property Name', 'Rating', 'Address', 'sector', 'Price', 'Area', 'Configuration', 'Floor', 'Facing', 'Overlooking', 'Property Age', 'URL', 'Nearby Places', 'details']]

In [142]:
df.sample(1)

,Property Name,Rating,Address,sector,Price,Area,Configuration,Floor,Facing,Overlooking,Property Age,URL,Nearby Places,details
43,Birla Navya,3.9,"Birla Navya\nSector 63A, Gurgaon",63,"₹ 3.1 Crore+ Govt Charges & Tax\n@ 16,315 per sq.ft. (All inclusive, Negotiable)",Super Built up area 1900\n(176.52 sq.m.) \nsq.ft.,"3 Bedrooms , 3 Bathrooms, 3 Balconies",2nd of 4 Floors,East,"Park/Garden,Main Road",0 to 1 Year Old,https://www.99acres.com/3-bhk-bedroom-independent-builder-floor-for-sale-in-birla-navya-sector-63a-gurgaon-1900-sq-ft-spid-Y81983100,['Sector 54 Chowk Metro Station\nWorldMark Mall Gurgaon\nBadshahpur Sohna Highway\nHeritage Xperiential Learning School'],"Property Ownership: :, Freehold | Flooring: :, Marble | Furnishing: :, Semifurnished | Width of facing road: :, 275.6 Feet | Corner Property: :, Yes | Parking: :, 2 Covered | Water Source: :, 24*7 Water | Power Backup: :, Full | Property Code: :, Y81983100"


In [143]:
df['Units'] = df['Price'].str.split(' ').str.get(2).str.split('\n').str.get(0).str.strip()

In [144]:
df['Units'].value_counts()

Units
Crore     5170
Crore+    3711
Lac        634
Lac+       572
Name: count, dtype: int64

**Price**

In [145]:
price_parts = df['Price'].str.split(' ')
numerical_value = price_parts.str.get(1).str.strip().astype(float)
unit = price_parts.str.get(2).str.split('\n').str.get(0).str.strip()

df['Price in Crore'] = np.where(
    unit.isin(['Lac', 'Lac+']),
    numerical_value / 100,
    numerical_value
)

In [146]:
df['Price in Crore'] = round(df['Price in Crore'], 2)

In [147]:
df.sample(1)

,Property Name,Rating,Address,sector,Price,Area,Configuration,Floor,Facing,Overlooking,Property Age,URL,Nearby Places,details,Units,Price in Crore
1753,DLF The Ultima,4.2,"DLF The Ultima\nSector 81, Gurgaon",81,"₹ 3.65 Crore+ Govt Charges & Tax\n@ 17,447 per sq.ft. (All inclusive) View Price Details",Super Built up area 2092\n(194.35 sq.m.)\nCarpet area: 1675 sq.ft. (155.61 sq.m.) \nsq.ft.,"3 Bedrooms , 3 Bathrooms, 3 Balconies with Servant Room",7th of 29 Floors,North-East,"Pool,Park/Garden,Club,Main Road,Others",1 to 5 Year Old,https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-sale-in-dlf-the-ultima-sector-81-gurgaon-2092-sq-ft-spid-X81666356,"['Vatika Town Square-INXT\nNational Highway 48\nEuro International School, Sec 84\nSGT University\nIndira Gandhi Intl Airport']","Transaction Type: :, Resale | Property Ownership: :, Freehold | Flooring: :, Wood | Furnishing: :, Semifurnished | Width of facing road: :, 32.0 Feet | Gated Community: :, Yes | Corner Property: :, Yes | Parking: :, 1 Open | WheelChair Friendly: :, Yes | Pet Friendly: :, Yes | Water Source: :, Municipal corporation,24*7 Water,Borewell/Tank | Power Backup: :, Full | Property Code: :, X81666356",Crore+,3.65


In [148]:
df.drop(columns= 'Units', inplace= True)

In [149]:
# Extract "Price per metric" (numeric value) and "Unit" (sq.ft., sq.m., etc.)
df[['Price_per_metric', 'Unit']] = df['Price'].str.extract(
    r'@\s*([\d,]+)\s*per\s*(sq\.ft\.|sq\.m\.|sq\.\s*yards?)',  # Regex pattern
    flags=re.IGNORECASE  # Case-insensitive matching
)

# Clean "Price_per_metric" (remove commas → convert to float)
df['Price_per_metric'] = df['Price_per_metric'].str.replace(',', '').astype(float)

# Handle cases where extraction failed (optional)
# df[['Price_per_metric', 'Unit']] = df[['Price_per_metric', 'Unit']].fillna('Not Specified')

In [150]:
# df[['Price in Crore', 'Price', 'Price_per_metric', 'Unit']]
df['Unit'].value_counts()

Unit
sq.ft.      9355
sq.yards     636
sq.m.         93
Name: count, dtype: int64

In [151]:
temp_df = df[df['Unit'].isnull()]
df = df[~(df['Unit'].isnull())]

In [152]:
# temp_df.iloc[0][['Price_per_metric', 'Unit']] = 19772, 'sq.ft'
# temp_df.iloc[1][['Price_per_metric', 'Unit']] = 8920, 'sq.ft'
# temp_df.iloc[2][['Price_per_metric', 'Unit']] = 8920, 'sq.ft'

# Faster for scalar values
temp_df.at[680, 'Price_per_metric'] = 19772
temp_df.at[680, 'Unit'] = 'sq.ft'
temp_df.at[3899, 'Price_per_metric'] = 8920
temp_df.at[3899, 'Unit'] = 'sq.ft'
temp_df.at[6942, 'Price_per_metric'] = 8920
temp_df.at[6942, 'Unit'] = 'sq.ft'

In [156]:
df = pd.concat([df, temp_df], axis= 0)

In [154]:
df['Price_per_sqft'] = df.apply(
    lambda row: (
        row['Price_per_metric'] * 9
        if 'yard' in row['Unit'].lower()
        else (
            row['Price_per_metric'] * 10.7639
            if 'sq.m' in row['Unit'].lower()
            else row['Price_per_metric']
        )
    ), axis= 1
)

df['Price_per_sqft'] = df['Price_per_sqft'].round()

In [159]:
num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 3788


In [160]:
duplicated_df = df[df.duplicated()]
duplicated_df_sorted = duplicated_df.sort_values(by=duplicated_df.columns.tolist())
duplicated_df_sorted.head(20)

,Property Name,Rating,Address,sector,Price,Area,Configuration,Floor,Facing,Overlooking,Property Age,URL,Nearby Places,details,Price in Crore,Price_per_metric,Unit,Price_per_sqft
541,18MTR ROAD LUXURY BUILDER FLOOR,NaN,"18MTR ROAD LUXURY BUILDER FLOOR\nDLF Phase 1, Gurgaon",NaN,"₹ 6.5 Crore\n@ 14,386 per sq.ft. (All inclusive, Negotiable) View Price Details",Carpet area: 4518\n(419.74 sq.m.) \nsq.ft.,"4 Bedrooms , 5 Bathrooms, 3+ Balconies with Pooja Room,Study Room,Servant Room,Others",2nd of 4 Floors,East,"Pool,Club,Others,Main Road,Park/Garden",1 to 5 Year Old,https://www.99acres.com/4-bhk-bedroom-independent-builder-floor-for-sale-in-dlf-phase-1-gurgaon-4518-sq-ft-spid-U82532266,['Sikandarpur metro station\nSikanderpur metro station\nSikandarpur rmrg metro station\nDlf phase 1 metro station\nGuru dronacharya metro station'],"Property Ownership: :, Freehold | Flooring: :, Marble | Furnishing: :, Semifurnished | Width of facing road: :, 39.4 Feet | Gated Community: :, Yes | Corner Property: :, Yes | Parking: :, 3 Covered, 1 Open | WheelChair Friendly: :, Yes | Pet Friendly: :, Yes | Water Source: :, Municipal corporation,Borewell/Tank,24*7 Water | Power Backup: :, Full | Property Code: :, U82532266",6.50,14386.0,sq.ft.,14386.0
672,18MTR ROAD LUXURY BUILDER FLOOR,NaN,"18MTR ROAD LUXURY BUILDER FLOOR\nDLF Phase 1, Gurgaon",NaN,"₹ 6.5 Crore\n@ 14,386 per sq.ft. (All inclusive, Negotiable) View Price Details",Carpet area: 4518\n(419.74 sq.m.) \nsq.ft.,"4 Bedrooms , 5 Bathrooms, 3+ Balconies with Pooja Room,Study Room,Servant Room,Others",2nd of 4 Floors,East,"Pool,Club,Others,Main Road,Park/Garden",1 to 5 Year Old,https://www.99acres.com/4-bhk-bedroom-independent-builder-floor-for-sale-in-dlf-phase-1-gurgaon-4518-sq-ft-spid-U82532266,['Sikandarpur metro station\nSikanderpur metro station\nSikandarpur rmrg metro station\nDlf phase 1 metro station\nGuru dronacharya metro station'],"Property Ownership: :, Freehold | Flooring: :, Marble | Furnishing: :, Semifurnished | Width of facing road: :, 39.4 Feet | Gated Community: :, Yes | Corner Property: :, Yes | Parking: :, 3 Covered, 1 Open | WheelChair Friendly: :, Yes | Pet Friendly: :, Yes | Water Source: :, Municipal corporation,Borewell/Tank,24*7 Water | Power Backup: :, Full | Property Code: :, U82532266",6.50,14386.0,sq.ft.,14386.0
892,18MTR ROAD LUXURY BUILDER FLOOR,NaN,"18MTR ROAD LUXURY BUILDER FLOOR\nDLF Phase 1, Gurgaon",NaN,"₹ 6.5 Crore\n@ 14,386 per sq.ft. (All inclusive, Negotiable) View Price Details",Carpet area: 4518\n(419.74 sq.m.) \nsq.ft.,"4 Bedrooms , 5 Bathrooms, 3+ Balconies with Pooja Room,Study Room,Servant Room,Others",2nd of 4 Floors,East,"Pool,Club,Others,Main Road,Park/Garden",1 to 5 Year Old,https://www.99acres.com/4-bhk-bedroom-independent-builder-floor-for-sale-in-dlf-phase-1-gurgaon-4518-sq-ft-spid-U82532266,['Sikandarpur metro station\nSikanderpur metro station\nSikandarpur rmrg metro station\nDlf phase 1 metro station\nGuru dronacharya metro station'],"Property Ownership: :, Freehold | Flooring: :, Marble | Furnishing: :, Semifurnished | Width of facing road: :, 39.4 Feet | Gated Community: :, Yes | Corner Property: :, Yes | Parking: :, 3 Covered, 1 Open | WheelChair Friendly: :, Yes | Pet Friendly: :, Yes | Water Source: :, Municipal corporation,Borewell/Tank,24*7 Water | Power Backup: :, Full | Property Code: :, U82532266",6.50,14386.0,sq.ft.,14386.0
1023,18MTR ROAD LUXURY BUILDER FLOOR,NaN,"18MTR ROAD LUXURY BUILDER FLOOR\nDLF Phase 1, Gurgaon",NaN,"₹ 6.5 Crore\n@ 14,386 per sq.ft. (All inclusive, Negotiable) View Price Details",Carpet area: 4518\n(419.74 sq.m.) \nsq.ft.,"4 Bedrooms , 5 Bathrooms, 3+ Balconies with Pooja Room,Study Room,Servant Room,Others",2nd of 4 Floors,East,"Pool,Club,Others,Main Road,Park/Garden",1 to 5 Year Old,https://www.99acres.com/4-bhk-bedroom-independent-builder-floor-for-sale-in-dlf-phase-1-gurgaon-4518-sq-ft-spid-U82532266,['Sikandarpur metro station\nSikanderpur metro station\nSikandarpur rmrg metro station\nDlf phase 